# Load MiraData Video Dataset from Hugging Face

Import the MiraData video dataset sample from GitHub into Pixeltable tables. MiraData is a high-quality video dataset with detailed annotations for video understanding tasks.

**What's in this recipe:**
- Import 100 sample videos from MiraData via CSV
- Load video clips with rich metadata (captions, timestamps, source info)
- Work with diverse video content from YouTube and other sources
- Leverage detailed annotations for video understanding tasks


## Problem

MiraData is a comprehensive video dataset with detailed annotations for video understanding. You need a representative sample in Pixeltable to apply AI models for video analysis, captioning, or understanding without downloading the massive full dataset from Hugging Face.


## Solution

**What's in this recipe:**
- Load 100 video samples from MiraData's curated sample CSV on GitHub
- Rich metadata including video URLs, captions, timestamps, and source information
- Diverse content from YouTube including 3D engine-rendered scenes and real-world footage
- Download videos from YouTube URLs using yt-dlp for analysis

The MiraData team provides a [sample CSV with 100 videos](https://github.com/mira-space/MiraData/blob/v1/assets/miradata_v1_100_samples.csv) that you can load directly into Pixeltable without needing to download the full dataset from [Hugging Face](https://huggingface.co/datasets/TencentARC/MiraData).

**Note:** The CSV contains YouTube URLs. To work with the actual video content, you'll need to download videos using a tool like `yt-dlp`.


### Setup


In [6]:
!uv add pixeltable yt-dlp

Resolved 262 packages in 4ms
Audited 165 packages in 26ms


In [7]:
import pixeltable as pxt

### Load MiraData Sample CSV

Load the [MiraData sample CSV](https://raw.githubusercontent.com/mira-space/MiraData/v1/assets/miradata_v1_100_samples.csv) containing 100 video samples with metadata.


In [8]:
# Load MiraData sample CSV from GitHub directly into Pixeltable
csv_url = "https://raw.githubusercontent.com/mira-space/MiraData/v1/assets/miradata_v1_100_samples.csv"

In [9]:
# Create directory and table for MiraData videos
pxt.drop_dir('miradata_videos', force=True)
pxt.create_dir('miradata_videos')

# Create table and load CSV data directly
t = pxt.create_table('miradata_videos.samples', source=csv_url)

Created directory 'miradata_videos'.
Created table 'samples'.
Inserting rows into `samples`: 100 rows [00:00, 4028.68 rows/s]
Inserted 100 rows with 0 errors.


### View Sample Data


In [10]:
# View sample data with key columns
t.select(t.video_url, t.source, t.short_caption, t.width, t.height).head(10)


video_url,source,short_caption,width,height
https://www.youtube.com/watch?v=AG2NSO20zkI,"youtube, 3D engine-rendered scenes",A person riding a motorcycle in a video game.,1280,720
https://www.youtube.com/watch?v=AG2NSO20zkI,"youtube, 3D engine-rendered scenes",A group of people walking down a street in a video game.,1280,720
https://www.youtube.com/watch?v=-iYMXRQCqCw,"youtube, 3D engine-rendered scenes",A person walking down a cobblestone path in a video game.,1280,720
https://www.youtube.com/watch?v=ViZdX5E3mRA,"youtube, 3D engine-rendered scenes",A person is walking through an old castle with stone walls and arches.,1280,720
https://www.youtube.com/watch?v=5I2SnLsGWhY,"youtube, 3D engine-rendered scenes",A person is walking through a forest in a video game.,1280,720
https://www.youtube.com/watch?v=tyqHqPm3upg,"youtube, 3D engine-rendered scenes",A video game character walking down a snowy street.,1280,720
https://www.youtube.com/watch?v=z0Z9GgEja8k,"youtube, 3D engine-rendered scenes",A video game with a gun shooting at a building.,1280,720
https://www.youtube.com/watch?v=_4U-sDY40Ug,"youtube, 3D engine-rendered scenes",The player is in a post-apocalyptic world with snow and ruins of buildings.,1280,720
https://www.youtube.com/watch?v=PjNpW2k2D1Y,"youtube, 3D engine-rendered scenes",The player is driving a car in a racing game and competing with other cars on an icy track.,1280,720
https://www.youtube.com/watch?v=cYgdn_aG2IU,"youtube, 3D engine-rendered scenes",A star wars character standing on a beach with palm trees in the background.,1280,720


In [ ]:
# Check total count
t.count()


In [ ]:
# View all available columns
t.select().head(3)


In [ ]:
### Download Videos from YouTube URLs (Optional)

To work with actual video content, we can create a UDF that downloads videos from YouTube URLs using yt-dlp and adds them as a computed column.

**Note:** Make sure to run the Setup cell above to install `yt-dlp` first.


In [11]:
import yt_dlp
import os

@pxt.udf
def download_youtube_video(video_url: str) -> pxt.Video:
    """Download a YouTube video and return the local file path as Video type"""
    if not video_url or 'youtube.com' not in video_url:
        return None
    
    # Create a persistent directory for downloads in the project
    download_dir = os.path.expanduser('~/.pixeltable/miradata_videos')
    os.makedirs(download_dir, exist_ok=True)
    output_template = os.path.join(download_dir, '%(id)s.%(ext)s')
    
    ydl_opts = {
        'format': 'best[ext=mp4]',
        'outtmpl': output_template,
        'quiet': True,
        'no_warnings': True,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            video_id = info['id']
            ext = info['ext']
            video_path = os.path.join(download_dir, f'{video_id}.{ext}')
            return video_path
    except Exception as e:
        print(f"Failed to download {video_url}: {e}")
        return None


In [12]:
# Add computed column to download first 10 videos only
# Using limit to keep the dataset size manageable (~500MB-1GB instead of 10GB+)
from pixeltable.iterators import RowIterator

# Get the first 10 rows' IDs
first_10_rows = t.select(t._rowid).limit(10).collect()
row_ids = [row['_rowid'] for row in first_10_rows]

# Add computed column only for these rows
t.add_computed_column(video=download_youtube_video(t.video_url))

ImportError: cannot import name 'RowIterator' from 'pixeltable.iterators' (/Users/alison-pxt/Documents/Github/pxt-cloud-sets/.venv/lib/python3.11/site-packages/pixeltable/iterators/__init__.py)

In [3]:
pxt.list_tables()

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/alison-pxt/.pixeltable/pgdata


['test_json',
 'test',
 'test2',
 'test3',
 'test_transcribe',
 'jane_goodall_video.clips',
 'jane_goodall_video.frames',
 'kamon_db.images',
 'kamon_db.translations',
 'kamon_db.parsed_raw',
 'kamon_db.kamon_images',
 'kamon_db.edo_images',
 'kamon_db.wiki_images',
 'kamon_db.kamon_gpt',
 'scene_view_demo.samples',
 'video_scene_demo.samples',
 'profile_processing.profiles',
 'mnist_data.digits',
 'image_demo.gray',
 'coco_images.mini_2017',
 'demo_project.lyrics',
 'padoru_images.samples',
 'cifar10_images.samples',
 'caption_contest.explanations',
 'miradata_videos.samples',
 'drone_audio.samples']

In [4]:
t = pxt.get_table('miradata_videos.samples')

In [5]:
t.head()

clip_id,source,video_url,video_id,width,height,fps,seconds,timestamp,frame_number,framestamp,file_path,short_caption,dense_caption,background_caption,main_object_caption,style_caption,camera_caption
0.4,"youtube, 3D engine-rendered scenes",https://www.youtube.com/watch?v=AG2NSO20zkI,AG2NSO20zkI,1280,720,30.,80.833,"['00:06:21.300', '00:07:42.133']",2425,"[11439, 13864]",video_clips/000000000/000000000000.4.mp4,A person riding a motorcycle in a video game.,"A virtual motorcycle ride through a bustling urban environment in a video game. The rider, clad in a yellow jacket, navigates the motorcycle through various cityscapes, including back alleys, main streets, and highways. The city is rich in detail, with dynamic weather conditions and a diverse array of pedestrians and vehicles populating the streets. The journey captures the essence of a high-speed chase or a time-sensitive mission, with the rider skillfully maneuvering around obstacles and traffic.","A detailed urban environment that transitions from a grimy alleyway to vibrant city streets and then to a highway. The city is alive with activity; pedestrians are walking, and vehicles are driving alongside the rider. The weather appears overcast, with wet roads reflecting the urban lights, hinting at recent rain. The time seems to be during the day, with sufficient ambient light illuminating the city's architecture and billboards.","A person wearing a yellow jacket, is riding a motorcycle. They start in a narrow alley, accelerate onto a busier city street, and continue through a densely populated urban area with pedestrians and traffic. The rider's movements are fluid and responsive, suggesting a gameplay mechanic that prioritizes speed and agility. The motorcycle's position shifts from the alley to the main road and then onto a highway, indicating a progression through different zones of the city.","The visual, photographic, and artistic style of the video is realistic with a touch of hyperrealism, emphasizing vivid colors, dynamic lighting, and a detailed environment to create an immersive gaming experience.","The view shot starts with a third-person perspective from behind the motorcycle, maintaining this angle throughout the video. The camera follows the rider closely, allowing for a clear view of the immediate surroundings and the road ahead. As the motorcycle accelerates, the camera seems to slightly shake, enhancing the sense of speed. There are no drastic changes in shooting angles, providing a consistent and immersive experience."
0.5,"youtube, 3D engine-rendered scenes",https://www.youtube.com/watch?v=AG2NSO20zkI,AG2NSO20zkI,1280,720,30.,79.267,"['00:07:42.133', '00:09:01.400']",2378,"[13864, 16242]",video_clips/000000000/000000000000.5.mp4,A group of people walking down a street in a video game.,"A dynamic urban environment within a video game, where a group of virtual characters is seen traversing a street that exhibits signs of dilapidation and abandonment. The atmosphere is heavy with a sense of dystopia, as the weather is overcast with rain, adding a layer of gloom to the already desolate cityscape. The architecture is a mix of futuristic structures and neglected buildings, hinting at a world that has seen better days. The characters move through this setting, interacting with each other and the surroundings, while vehicles navigate the wet roads, contributing to the sense of ongoing life in this virtual world.","A rich tapestry of a futuristic yet run-down city with a blend of advanced and decaying architecture. Palm trees and overgrown foliage suggest a once-thriving cityscape now in decline. The weather is consistently rainy, creating slick surfaces that reflect the sparse lighting from the buildings and street lamps. The time appears to be either dawn or dusk, contributing to the low-light conditions that pervade the scene.","The main subjects, a group of virtual characters, are depicted engaging in various activities as they move through the street. They wa

### Publish to Pixeltable Cloud

Publish the table to make it available on Pixeltable Cloud.


In [ ]:
# Publish the table to Pixeltable Cloud
pxt.publish(
    'miradata_videos.samples',
    'pxt://pixeltable:huggingface/miradata_videos',
    access='public'
)


## See also

- [MiraData on Hugging Face](https://huggingface.co/datasets/TencentARC/MiraData)
- [MiraData GitHub Repository](https://github.com/mira-space/MiraData)
- [MiraData Sample CSV](https://raw.githubusercontent.com/mira-space/MiraData/v1/assets/miradata_v1_100_samples.csv)
